# OPEN FOOD NOTEBOOK

In [131]:
import pandas as pd
import numpy as np
import scipy as sp
#import folium
import re
%matplotlib inline

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import seaborn as sns
import time

#from google.cloud import translate
#import pycountry
#import emoji

#translate_client = translate.Client()

import sys # for printing process
import unidecode # for normalizing text
from pathlib import Path # check files


from py_translator import Translator
translator = Translator()

# from googletrans import Translator

# WE HAD DIFFICULTIES MERGING, FATS CLEANING IS IN `project_fats` FILE

We need to create a data folder where the .csv file will be stored and also a maps folder where .html maps will be stored

In [133]:
filename = 'en.openfoodfacts.org.products.csv'
countryfile = 'wikipedia-iso-country-codes.csv'
translationsfile = 'translations.csv'
foodfile = 'food.csv'

In [123]:
using_col = [
    "product_name",
    "generic_name",
    "quantity",
    "brands",
    "brands_tags",
    "categories",
    "categories_tags",
    "categories_en",
    "manufacturing_places",
    "manufacturing_places_tags",
    "labels",
    "labels_tags",
    "labels_en",
    "purchase_places",
    "countries",
    "countries_tags",
    "countries_en",
    "ingredients_text",
    "allergens",
    "allergens_en",
    "traces",
    "traces_tags",
    "traces_en",
    "nutrition_grade_uk",
    "nutrition_grade_fr",
    "main_category",
    "main_category_en",
    "energy_100g",
    "energy-from-fat_100g",
    "fat_100g",
    "saturated-fat_100g",
    "trans-fat_100g",
    "cholesterol_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "sodium_100g",
    "alcohol_100g",
    "calcium_100g",
    "iron_100g",
    "carbon-footprint_100g",
    "nutrition-score-fr_100g",
    "nutrition-score-uk_100g",
    "glycemic-index_100g"
]

In [124]:
# CONSTANTS
# unknown values to use
UNKNOWN_NR='-1'
UNKNOWN_STR='unknown'
# delay between translation requests
TRANSLATION_DELAY=0.3
# progress in function
PROGRESS=0


In [132]:
# cache translations to save translation requests
translations_file = Path(data_folder + translationsfile)
translations = {}

if not translations_file.is_file():
    print('Translations file not found')
else:
    print('Translations file found')
    translations = pd.read_csv(data_folder + translationsfile, 
                               sep='\t',
                               low_memory=False).to_dict("records")[0]
    print('{} translations found'.format(len(translations)))

Translations file found
3026 translations found


In [ ]:
food_file = Path(data_folder + foodfile)
food_df = pd.DataFrame()

if not food_file.is_file():
    print('Food file not found')
    food_df = pd.read_csv(data_folder + filename, 
                      sep='\t',
                      header=0,
                      usecols = using_col,
                      quotechar='"', 
                      low_memory=False)
else:
    print('Food file found')
    food_df = pd.read_csv(data_folder + foodfile, 
                      sep='\t',
                      low_memory=False)
    print('{} Food entries found'.format(len(food_df)))


In [128]:
print("The types of the data set are: \n", format(food_df.dtypes))
print ("The total size of the data set is:", format(food_df.shape) )
food_df.head(5)

The types of the data set are: 
 product_name                  object
generic_name                  object
quantity                      object
brands                        object
brands_tags                   object
categories                    object
categories_tags               object
categories_en                 object
manufacturing_places          object
manufacturing_places_tags     object
labels                        object
labels_tags                   object
labels_en                     object
purchase_places               object
countries                     object
countries_tags                object
countries_en                  object
ingredients_text              object
allergens                     object
allergens_en                  object
traces                        object
traces_tags                   object
traces_en                     object
nutrition_grade_uk           float64
nutrition_grade_fr            object
main_category                 object
main_

,product_name,generic_name,quantity,brands,brands_tags,categories,categories_tags,categories_en,manufacturing_places,manufacturing_places_tags,...,proteins_100g,salt_100g,sodium_100g,alcohol_100g,calcium_100g,iron_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g
0,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.8,1.40,0.551181,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cacao,NaN,130 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sauce Sweety chili 0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.2,2.04,0.803150,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mendiants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Salade de carottes râpées,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9,0.42,0.165354,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Cleaning Data

In [137]:
# remove rows where the columns we are interested in are all null
food_df = food_df.dropna(subset=using_col, how='all')
print("Number of rows w/missing product_name: {}".format(len(food_df) - food_df['product_name'].count()))
saveFoodDF()

Number of rows w/missing product_name: 25881


# Helper Functions

In [239]:
def getValueWithPriorityColumns(input_row, merging_columns):
    for column in merging_columns:
        if pd.notnull(input_row[column]):
            return input_row[column]
    return input_row[merging_columns[0]]

def mergeColumnsFromDF(input_df, desired_columns, result_column):
    if result_column in input_df.columns:
        return input_df

    input_df[result_column] = input_df.apply(
        lambda x: getValueWithPriorityColumns(x,desired_columns),
        axis = 1
    )
    for column in desired_columns:
        if column in input_df.columns:
            input_df = input_df.drop(column, axis=1)

    return input_df


In [ ]:
def translateWithCache(value):
    global translations
    # search translated word in translations map
    if value in translations:
        # print("Cached  {} -> {}".format(value,translations[value]))
        return translations[value]
    else:
        try:
            time.sleep(TRANSLATION_DELAY) 
            trns_value = translator.translate(text=value, dest='en')
            if not trns_value is None:
                new_translation=trns_value.text.lower()
                # print("Translating {} -> {}".format(value,new_translation))
                translations[value]=new_translation
                return new_translation
            else:
                print("None {} / {} / {}".format(value,type(value), e))
                
        except Exception as e:
            print("Exception {} / {} / {}".format(value,type(value), e))
    return value

In [ ]:
# Parse and translate columns that are in the format "language:value"
def parseTranslate(x, target_columns):
    for column in target_columns:    
        if (':') in x[column]:
            info_ = x[column].split(':')
            if len(info_) == 2:
                value = info_[1]
                return translateWithCache(value)
        return x[column]

In [ ]:
def showProgress(size):
    global PROGRESS 
    PROGRESS += 1
    progress_value = int(100*PROGRESS/size)/100 
    if (progress_value*100)%1==0:
        sys.stdout.write('\r'+'Progress {}%'.format(100*progress_value))
        sys.stdout.flush()

In [136]:
def saveTranslations():
    pd.DataFrame.from_dict(translations,orient="index").T.to_csv(data_folder + translationsfile,sep='\t')
    
def saveFoodDF():
    food_df.to_csv(data_folder + foodfile,sep='\t')

def normalizeString(string):
    return unidecode.unidecode(string.lower().strip())

In [210]:
def showNanPercentage(df,desired_columns):
    for column in desired_columns:
        print("Percentage of nan in {} is {}%".format(column,100*len(df[df[column].isna()])/len(df) ))
        

In [248]:
def formatAndTranslateRow(row, translateNoFormat):
    showProgress(allergen_notna_df.shape[0])

    if type(row) is not list and pd.notnull(row) :
        raw_data = row.lower().split(',')
        data = []
        for value in raw_data:
            value_ = normalizeString(value)
            # format <langage_code:info>
            if (':') in value_:
                info_ = value_.split(':')
                if len(info_) == 2:
                    # already in english
                    if info_[0] == 'en':
                        data.append(info_[1])
                    # translate to english
                    else:
                        data.append(translateWithCache(info_[1]))
                else:
                    data.append(info_)
                #print("Appending {}".format(info_))
                
            # no format, let's translate it
            else:
                if translateNoFormat:
                    data.append(translateWithCache(value_))
                else:
                    data.append(value_)
        #print(data)
        return data                       
    else:
        return row
    


## Filling in missing product_name
This section deals with NaN values for `product_name`. If it does not have a `product_name`, the `generic_name` was used. If neither field was filled, then a combination of `brands` and `categories_en`/`main_category` was used.

In [ ]:
def get_name(row):
    if pd.isnull(row['product_name']):
        if pd.isnull(row['generic_name']):
            if pd.isnull(row['main_category_en']) & pd.isnull(row['categories_en']) & pd.isnull(row['brands']):
                return
            else:
                category_name = row['main_category_en']
                if pd.isnull(category_name):
                    category_name = row['categories_en']
                return "{} {}".format(row['brands'], category_name)
        else:
            return row['generic_name']
    else:
        return row['product_name']
    
df = food_df.copy()

df['product_name'] = df.apply(
    lambda x: get_name(x),
    axis = 1
)

In [ ]:
print("Number of rows w/missing product_name after modifications: {}".format(len(df) - df['product_name'].count()))

In [ ]:
# removing the columns that we no longer need
used_col = [
    'generic_name',
    'main_category',
    'main_category_en',
    'brands',
    'brands_tags',
    'categories',
    'categories_tags',
    'categories_en',
]

df = df.drop(used_col, axis=1)

As seen in the results, 22 458 rows still do not have names after our modifications. Our team decided that names were not of particular importance for our analysis, so we decided to leave these no-named items in the dataframe. The name is not important because we mostly want to analyze the ingredients of the items for each country. Thus the `labels`, `allergens`, and the numbers for sugar/sodium/calcium/etc and `countries` are the important columns.

## Filling in Missing Values for Country
This section deals with the missing values for `countries_en`. The `countries_en` column represents the countries where the product is sold. This column is important for our analysis because we want to analyze how viable it is to live in each country based off one's dietary restrictions.

In order to fix these missing values, we decided to first fill the column with values from `purchase_places`, then `manufacturing_places`. We decided to use `purchase_places` because if it was purchased in a certain country, obviously it means the product is sold there as well. As for `manufacturing_places`, we assumed that it is most likely that a product manufactured in a country would be sold there as well.

Furthermore, we looked at the column `origins`, however this column is actually describing where each ingredient came from. This would not be helpful for us because the origin would not tell us about which countries actually sell/consume this specific item.

In [ ]:
print("Number of rows w/missing countries_en: {}".format(len(df) - df['countries_en'].count()))

In [ ]:
df_1['countries_en'].count()

In [ ]:
def translate_country(row):
    if pd.isnull(row['countries_en']):
        alt_country = None
        if pd.notna(row['purchase_places']):
            alt_country = row['purchase_places']
        elif pd.notna(row['manufacturing_places']):
            alt_country = row['manufacturing_places']
            
        # got value from purchase_places or manufacturing_places
        if (not alt_country is None) and pd.notna(alt_country):
            try:
                en_alt_country = translator.translate(text=alt_country, dest='en')
                if not en_alt_country is None:
                    return en_alt_country.text
            except Exception as e:
                return alt_country
            
        return alt_country
    else:
        return row['countries_en']
    
df_1 = df.copy()

df_1['countries_en'] = df_1.apply(
    lambda x: translate_country(x),
    axis = 1
)

In [ ]:
print("Number of rows w/missing countries_en: {}".format(len(df_1) - df_1['countries_en'].count()))
print("Percentage of rows w/missing countries_en: {0:.3f}%".format(100*(len(df_1)-df_1['countries_en'].count())/len(df_1)))


In [ ]:
# drop rows without country
df_1 = df_1.dropna(subset=['countries_en'])


print("Number of rows w/multiple countries: {}".format(
    len(df_1[df_1['countries_en'].str.contains(',')])))

print("Number of total rows: {}".format(len(df_1)))

In [ ]:
# remove columns that are no longer needed
used_col = [
    'purchase_places',
    'manufacturing_places',
    'manufacturing_places_tags',
    'countries_tags',
    'countries',
]

df_1 = df_1.drop(used_col, axis = 1)

In [ ]:
# shows that some countries_en are lists
df_1[df_1['countries_en'].notnull() & df_1['countries_en'].str.contains(',')][['countries_en', 'product_name']].head()

In [ ]:
# converts the list substring into an array
df_2 = df_1.copy()

df_2['countries_en'] = df_2.apply(
    lambda x: [x.strip() for x in x['countries_en'].split(',')],
    axis = 1
)

# [x.strip() for x in my_string.split(',')]

In [ ]:
# shows that the countries has been properly split
df_2[df_2.index == 173][['product_name', 'countries_en']]

Next, this subsection deals with standardizing the countries for each product. First, we notice that some products have more than one country in their `countries_en` field. In this case, we seperate/explode each country in the `countries_en` field so that each country has its own row for that item. Next, we join the countries with their respective country code.

In [ ]:
# map the countries_en to country codes
country_df = pd.read_csv(data_folder + countryfile, 
                         sep=',',
                         header=0,
                         usecols=['English short name lower case', 'Alpha-2 code'],
                         quotechar='"')
# rename columns
country_df.rename(columns={
    'Alpha-2 code':'country_code',
    'English short name lower case': 'country_name'
    }, inplace=True)

country_df['country_name'] = country_df.apply(
    lambda x: x['country_name'].lower(),
    axis=1
)

country_df.head()

In [ ]:
def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

df_3 = df_2.copy()
df_3 = explode(df_3,'countries_en')

In [ ]:
# see how the explode function created another row because there were two countries for Lion Peanut x2
df_3[df_3['product_name'].notna() & df_3['product_name'].str.contains('Lion Peanut x2')]

In [ ]:
df_4 = df_3.copy()
# rename countries_en to country_name
df_4.rename(columns={'countries_en':'country_name'}, inplace=True)

df_4['country_name'] = df_4.apply(
    lambda x: x['country_name'].lower(),
    axis = 1
)

In [ ]:
# method to check the stats of the rows with a country_name but still without a country_code
def notAssigned(df_sample):
    not_assigned = df_sample[df_sample['country_name'].notna() & df_sample['country_code'].isna()]

    print("Number of unassigned items is: {}".format(len(not_assigned)))
    print("The important values are: ")
    print(not_assigned['country_name'].value_counts())

In [ ]:
df_5 = df_4.copy()

df_5 = df_5.merge(country_df, how='left')
    
notAssigned(df_5)

We decided to fix the country codes with the highest frequency, since the importance/effect of fixing the lower values will decrease as we descend through the list.

In [ ]:
# changing Russian Federation to russia
country_df['country_name'][country_df['country_code'] == 'RU'] = 'russia'

# changing Korea, Republic of to south korea
country_df['country_name'][country_df['country_code'] == 'KR'] = 'south korea'

# changing Macedonia, the former Yugoslav Republic of to republic of macedonia
country_df['country_name'][country_df['country_code'] == 'MK'] = 'republic of macedonia'

# changing Taiwan, Province of China to taiwan
country_df['country_name'][country_df['country_code'] == 'TW'] = 'taiwan'

# changing Viet Nam to vietnam
country_df['country_name'][country_df['country_code'] == 'VN'] = 'vietnam'


From the above analysis of the unpaired countries, we see that a few countries are still in another language. Specifically, they are in the format "language:country". The method `parseTranslate` tries to deal with this issue.

In [ ]:
df_6 = df_4.copy()

df_6['country_name'] = df_6.apply(
    lambda x: parseTranslate(x,['country_name']),
    axis = 1
)

From our description of the countries still missing country codes, it is found that most of these countries do not have the full name as the one in the CSV file `country_df`. Thus, we try to find the `best_match` and change the `country_name` in the food dataframe to match the one in the `country_df`. We consider something a `best_match` if the `country_name` from the food dataframe is a substring of the `country_name` in the country dataframe.

In [ ]:
df_6 = df_6.merge(country_df, how='left')

In [ ]:
# Display result
df_6[['product_name','country_name','country_code']].head()

In [ ]:
def best_match(country_df, row):
    if pd.isnull(row['country_code']):
        countries = list(country_df['country_name'])
        for country in countries:
            if row['country_name'] in country:
                return country

    return row['country_name']

df_6['country_name'] = df_6.apply(
    lambda x: best_match(country_df, x),
    axis = 1
)

In [ ]:
df_7 = df_6.drop(['country_code'], axis=1)

df_7 = df_7.merge(country_df, how='left')

In [ ]:
notAssigned(df_7)

In [ ]:
df_8 = df_7[df_7['country_code'].notna()]
print("Number of rows with a country code: {}".format(len(df_8)))
print("Number of total rows: {}".format(len(food_df)))

The number of rows we can use (meaning the rows with a `country_code`) is higher than what we original started with because we made duplicates of some rows so that each country has its own instance of the item. An issue we ran into is that with the high number of translations we need to do, Google's API will eventually block our requests,thus some more rows might have actually been able to be paired up with a `country_code`. To try a walka

## Filling in missing nutrition scores

This section deals with NaN values for `nutrition score`.

In [ ]:
desired_columns=[
    'nutrition_grade_uk',
    'nutrition_grade_fr',
    'nutrition-score-fr_100g',
    'nutrition-score-uk_100g'
]
result_column='nutrition_score'

Starting with the analysis let's show the percentage of nan values in the desired columns

In [ ]:
for column in desired_columns:
    print("Percentage of nan in {} is {}%".format(column,100*len(food_df[food_df[column].isna()])/len(food_df) ))

It's important to note that `nutrition_grade_uk` is always nan and that `nutrition_grade_fr`, `nutrition-score-fr_100g` and `nutrition-score-uk_100g` have exactly the same value. For this reason, the column used is `nutrition_grade_fr`. Nan values are not filled since for now, the nutrition score is going to be an additional indicator.

In [ ]:
nutrition_df = food_df.copy()
nutrition_df[result_column]=nutrition_df[desired_columns[1]] #.fillna(UNKNOWN_STR)
nutrition_df = nutrition_df.drop(desired_columns, axis=1)

In [ ]:
nutrition_df[result_column].head()

## Filling in missing allergens

This section deals with NaN values for `allergens`.

In [213]:
desired_columns=[
    'allergens_en',
    'allergens'
]
result_column='allergen_values'
allergen_df=food_df.copy()

Starting with the analysis let's show the percentage of nan values in the desired columns

In [ ]:
showNanPercentage(food_df,desired_columns)
allergen_df[allergen_df[desired_columns[0]].notna()][desired_columns].head(5)

Looking at the result, both columns have different percentages, for some reason, values in `allergens_en` are urls so for further analysis only the `allergen` column is taken.

In [181]:
allergen_df[result_column]=allergen_df[desired_columns[1]]
allergen_df1 = allergen_df.drop(desired_columns, axis=1)

Looking at the allergens format:

In [ ]:
allergen_notna_df=allergen_df1[allergen_df1[result_column].notna()].copy()
allergen_notna_df[result_column].head()

Pre-processing of the `result column` by assuring values are strings lowercase before processing

In [ ]:
allergen_notna_df[result_column].apply(str)
allergen_notna_df[result_column] = allergen_notna_df[result_column].apply(
    lambda x: x.lower()
)

Using the helper function `formatAndTranslateRow`, allergen rows are going to be formated to an array and translated to english

In [111]:
PROGRESS=0
allergen_notna_df[result_column] = allergen_notna_df[result_column].apply(
    lambda x: formatAndTranslateRow(x,True)
)
saveTranslations()

Progress 100.0%

In [141]:
print(allergen_notna_df.shape[0])
allergen_notna_df[allergen_notna_df[result_column].notna()][result_column].head(10)

68868


10                                       [eggs, mustard]
22     [corn, gluten, became, cheese, milk, milk, mil...
31                [corn, rye, corn, rye, salmon, cheese]
39                                              [cheese]
44     [corn, gluten, became, became, barley, corn, s...
46                                        [eggs, gluten]
282                                                [soy]
293                       [corn, butter, eggs, hazelnut]
315                                   [almonds, almonds]
342                                     [milk, hazelnut]
Name: allergen_values, dtype: object

In [193]:
allergen_df1_=allergen_df1.rename(columns = {result_column:'old_values'})['old_values']
allergen_df1[result_column]=pd.concat([allergen_df1_, allergen_notna_df], axis=1, join_axes=[allergen_df1.index])[result_column]           


In [215]:
print(allergen_df1.shape)
allergen_df1[allergen_df1[result_column].notna()][result_column].head(10)

(693846, 45)


10                                       [eggs, mustard]
22     [corn, gluten, became, cheese, milk, milk, mil...
31                [corn, rye, corn, rye, salmon, cheese]
39                                              [cheese]
44     [corn, gluten, became, became, barley, corn, s...
46                                        [eggs, gluten]
282                                                [soy]
293                       [corn, butter, eggs, hazelnut]
315                                   [almonds, almonds]
342                                     [milk, hazelnut]
Name: allergen_values, dtype: object

In [216]:
showNanPercentage(allergen_df1,[result_column])

Percentage of nan in allergen_values is 90.07445456196332%


## Filling in traces

This section deals with NaN values for `traces`.

In [242]:
desired_columns=[
    'traces_en',
    'traces'
]
result_column='traces_values'
traces_df=food_df.copy()

Starting with the analysis let's show the percentage of nan values in the desired columns

In [243]:
traces_df_=mergeColumnsFromDF(traces_df, desired_columns, result_column)
traces_notna_df=traces_df_[traces_df_[result_column].notna()].copy()

In [247]:
showNanPercentage(traces_df_,[result_column])
traces_notna_df[result_column].head()

Percentage of nan in traces_values is 91.46035287369243%


111                                            Eggs,Milk
129                                         Sesame seeds
220    Eggs,Gluten,Milk,Nuts,Soybeans,Oatmeal,Wheatflour
255    fr:contient-oeuf-lait-anchois-soya-ble-seigle-...
275    Soybeans,Sulphur dioxide and sulphites,fr:cont...
Name: traces_values, dtype: object

Translate

In [253]:
PROGRESS=0
traces_notna_df[result_column] = traces_notna_df[result_column].apply(
    lambda x: formatAndTranslateRow(x,False)
)
saveTranslations()

Progress 4.0%Exception arete / <class 'str'> / ('Connection aborted.', OSError("(60, 'ETIMEDOUT')",))
Progress 4.0%Exception du-celeri-et-du-sesame / <class 'str'> / Expecting value: line 1 column 1 (char 0)
Exception fabrique-dans-un-atelier-qui-utilise-des-fruits-a-coques / <class 'str'> / Expecting value: line 1 column 1 (char 0)
Progress 4.0%Exception de-lupin-et-de-graines-de-sesames / <class 'str'> / Expecting value: line 1 column 1 (char 0)
Progress 4.0%Exception huile-de-palme / <class 'str'> / Expecting value: line 1 column 1 (char 0)
Progress 4.0%Exception noix-de-coco / <class 'str'> / ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
Progress 23.0%Exception sesae / <class 'str'> / ('Connection aborted.', OSError("(65, 'EHOSTUNREACH')",))
Progress 86.0%

In [254]:
print(traces_notna_df.shape[0])
traces_notna_df[traces_notna_df[result_column].notna()][result_column].head(10)

59252


111                                         [eggs, milk]
129                                       [sesame seeds]
220    [eggs, gluten, milk, nuts, soybeans, oatmeal, ...
255    [contains egg-milk-anchovy-soy-wheat-rye-barle...
275    [soybeans, sulphur dioxide and sulphites, cont...
286    [gluten, may contain nuts, soy-varied-milk-egg...
293                       [nuts, sesame seeds, soybeans]
299    [celery, crustaceans, eggs, fish, gluten, milk...
300                                               [eggs]
306    [eggs, gluten, milk, mustard, nuts, sesame see...
Name: traces_values, dtype: object

## Filling in Ingredients
This section deals with NaN values for `ingredients`.

In [260]:
desired_columns=[
    'ingredients_text'
]
result_column='ingredients_text'
ingredients_df=food_df.copy()

Starting with the analysis let's show the percentage of nan values in the desired columns

In [261]:
showNanPercentage(food_df,desired_columns)
ingredients_df[ingredients_df[desired_columns[0]].notna()][desired_columns].head(5)

Percentage of nan in ingredients_text is 43.29880117490048%


,ingredients_text
10,"antioxydant : érythorbate de sodium, colorant ..."
15,"Lait entier, sucre, amidon de maïs, cacao, Aga..."
22,"baguette Poite vin Pain baguette 50,6%: fqrine..."
31,"Paln suédois 42,6%: farine de BLÉ, eau, farine..."
33,"Taboulé 76,2%, légumes 12%, huile de colza, se..."


In [271]:
ingredients_notna_df=ingredients_df[ingredients_df[result_column].notna()].copy()
ingredients_notna_df[result_column].loc[10]

"antioxydant : érythorbate de sodium, colorant : caramel - origine UE), tomate 33,3%, MAYONNAISE 11,1% (huile de colza 78,9%, eau, jaunes d'OEUF 6%, vinaigre, MOUTARDE [eau, graines de MOUTARDE, sel, vinaigre, curcuma], sel, dextrose, stabilisateur : gomme de cellulose, conservateur : sorbate de potassium, colorant : ?-carotène, arôme)"

In [270]:
ingredients_notna_df[result_column].loc[10].split(':')

['antioxydant ',
 ' érythorbate de sodium, colorant ',
 " caramel - origine UE), tomate 33,3%, MAYONNAISE 11,1% (huile de colza 78,9%, eau, jaunes d'OEUF 6%, vinaigre, MOUTARDE [eau, graines de MOUTARDE, sel, vinaigre, curcuma], sel, dextrose, stabilisateur ",
 ' gomme de cellulose, conservateur ',
 ' sorbate de potassium, colorant ',
 ' ?-carotène, arôme)']

In [ ]:
PROGRESS=0
ingredients_notna_df[result_column] = ingredients_notna_df[result_column].apply(
    lambda x: formatAndTranslateRow(x,True)
)
saveTranslations()

In [ ]:
print(ingredients_notna_df.shape[0])
traces_notna_df[ingredients_notna_df[result_column].notna()][result_column].head(10)

# Data Visualization & Analysis
TO BE COMPLETED FOR MILESTONE 3

In [257]:
len(translations)

5805

In [ ]:
saveTranslations()